In [1]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pathlib
import synapseclient

# custom package(s)
from utils import utils, synapse_login

import yaml
import pathlib

with open("./local_configs/notebook_config.yaml", "r") as f:
    config = yaml.safe_load(f)

csv_model = pathlib.Path("../" + config["file_names"]["csv_model"]).resolve()
json_model = pathlib.Path("../" + config["file_names"]["json_model"]).resolve()

syn = synapse_login.main()

Welcome, Nicholas Lee!



In [2]:
df = pd.read_csv('../EL.data.model.csv')

In [3]:
studies = syn.tableQuery("select * FROM syn51209786")
studies = studies.asDataFrame().fillna("")

In [6]:
studies

,name,shortName,program,principalInvestigators,grantNumber,grantSerialNumber,abstract,institutions,externalWebsite,link,isFeatured,moreInformation,moreInformationURL,computationTools
1_18,The Long Life Family Study,LLFS,ELITE,"[Michael Province , Kaare Christensen, Stephan...",[U19AG063893],AG063893,The Long Life Family Study (LLFS) has enrolled...,"[Washington University, University of Southern...",https://longlifefamilystudy.wustl.edu/LLFS/Hom...,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,[]
2_18,The Longevity Genomics Consortium,LG,ELITE,"[Steve Cummings, Nicholas Schork, Simon Melov,...",[U24AG051129],AG051129,"Human longevity is heritable, and statisticall...","[California Pacific Medical Center, Buck Insti...",https://www.longevitygenomics.org/index.php/,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,"[syn26433712, syn26483536, syn26483537, syn264..."
3_23,The Longevity Consortium,LC,ELITE,"[Steve Cummings, Nicholas Schork]",[U19AG023122],AG023122,THE LONGEVITY CONSORTIUM (LC). Multiple studie...,"[California Pacific Medical Center, Translatio...",https://www.longevityconsortium.org/home,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,"[syn26483536, syn26483537, syn26483538, syn264..."
4_22,The Integrative Longevity Omics Projects,ILO,ELITE,"[Thomas Perls , Paola Sebastiani]","[UH2AG064704, UH3AG064706]",AG064704,"Centenarians (ages >100 yrs) and even more-so,...","[Boston University, Tufts Medical Center]",https://longevityomics.com/,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,[]
5_18,Integrative Omics to enhance therapeutics deve...,,ELITE,"[Nicholas Schork, Richard Miller]",[UH2AG064706],AG064706,A number of drugs and interventions have been ...,"[Translational Genomics Research Institute, Un...",https://longevityomics.com/,https://projectreporter.nih.gov/project_info_d...,False,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,[]


In [4]:
projects = syn.tableQuery("select * from syn52168034")
projects = projects.asDataFrame()

In [7]:
projects.reset_index(inplace=True)

In [9]:
projects["name"] = projects["name"].str.replace("EL-LC", "LC")

In [12]:
projects2 = projects.merge(
    studies, how='left', left_on='name', right_on='shortName')

In [14]:
projects2.columns

Index(['index', 'id', 'name_x', 'description', 'createdOn', 'createdBy',
       'etag', 'modifiedOn', 'modifiedBy', 'portal', 'name_y', 'shortName',
       'program', 'principalInvestigators', 'grantNumber', 'grantSerialNumber',
       'abstract', 'institutions', 'externalWebsite', 'link', 'isFeatured',
       'moreInformation', 'moreInformationURL', 'computationTools'],
      dtype='object')

In [15]:
projects2[["name_x", "name_y"]]

,name_x,name_y
0,LLFS,The Long Life Family Study
1,LG,The Longevity Genomics Consortium
2,ILO,The Integrative Longevity Omics Projects
3,LC,The Longevity Consortium


In [22]:
projects2 = projects2.rename(columns={'name_y': 'longName', 'name_x': 'name'})

In [84]:
projects2 = projects2.drop(columns=['index', 'description', 'name', 'createdOn',
                           'createdBy', 'abstract', 'etag', 'modifiedOn', 'modifiedBy'], errors='ignore')

In [85]:
projects2

,id,portal,longName,shortName,program,principalInvestigators,grantNumber,grantSerialNumber,institutions,externalWebsite,link,isFeatured,moreInformation,moreInformationURL,computationTools
0,syn52072575,ELITE,The Long Life Family Study,LLFS,ELITE,"[Michael Province , Kaare Christensen, Stephan...",[U19AG063893],AG063893,"[Washington University, University of Southern...",https://longlifefamilystudy.wustl.edu/LLFS/Hom...,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,[]
1,syn52072939,ELITE,The Longevity Genomics Consortium,LG,ELITE,"[Steve Cummings, Nicholas Schork, Simon Melov,...",[U24AG051129],AG051129,"[California Pacific Medical Center, Buck Insti...",https://www.longevitygenomics.org/index.php/,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,"[syn26433712, syn26483536, syn26483537, syn264..."
2,syn52237024,ELITE,The Integrative Longevity Omics Projects,ILO,ELITE,"[Thomas Perls , Paola Sebastiani]","[UH2AG064704, UH3AG064706]",AG064704,"[Boston University, Tufts Medical Center]",https://longevityomics.com/,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,[]
3,syn52642213,ELITE,The Longevity Consortium,LC,ELITE,"[Steve Cummings, Nicholas Schork]",[U19AG023122],AG023122,"[California Pacific Medical Center, Translatio...",https://www.longevityconsortium.org/home,https://projectreporter.nih.gov/project_info_d...,True,Exceptional Longevity Translational Resources,https://help.adknowledgeportal.org/apd/Exeptio...,"[syn26483536, syn26483537, syn26483538, syn264..."


In [86]:
for n in projects2.to_dict('records'):
    syn_id = n.pop('id')
    for k, v in n.items():
        if type(v) != list:
            n[k] = [v]
        elif type(v) == list and len(v) == 0: 
            n[k] = ['']
        
    e = syn.get_annotations(syn_id)
    new_annos = synapseclient.Annotations(syn_id, e._etag, n)
    e = syn.set_annotations(new_annos)

In [63]:
new_annos

{'portal': ['ELITE'],
 'longName': ['The Longevity Consortium'],
 'shortName': ['LC'],
 'program': ['ELITE'],
 'principalInvestigators': ['Steve Cummings', 'Nicholas Schork'],
 'grantNumber': ['U19AG023122'],
 'grantSerialNumber': ['AG023122'],
 'abstract': ['THE LONGEVITY CONSORTIUM (LC). Multiple studies of model organisms and humans have suggested that genetic variants, proteins, metabolites, as well as other biomolecules and molecular- physiologic processes, could play roles in mediating longevity, healthspan, and age-related disease in humans. Unfortunately, the direct relevance of many of these factors to human longevity, healthspan and age-related disease is uncertain, as is the amenability of these factors to pharmacological modulation. Therefore, well-integrated and sophisticated research strategies are needed to determine the degree to which various factors both influence human longevity and are amenable to pharmacological manipulation. The Longevity Consortium (LC) has a his

In [1]:
import synapseclient

syn = synapseclient.login()


UPGRADE AVAILABLE

A more recent version of the Synapse Client (4.1.1) is available. Your version (3.1.1) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 4.1.1 release notes

https://python-docs.synapse.org/news/



Welcome, Nicholas Lee!



In [7]:
annotations = syn.tableQuery("SELECT * FROM syn51209885.2 WHERE studyKey = 'ELPSCRNA'")

In [8]:
annotations = annotations.asDataFrame()

In [11]:
annotations = annotations.fillna('')

In [22]:
new_annos = annotations.to_dict("records")[0]

In [25]:
new_annos['relatedStudies'] = [s.strip() for s in new_annos['relatedStudies']]

In [27]:
new_annos["studyWiki"] = "syn53366748"

In [30]:
new_annos

{'studyKey': 'ELPSCRNA',
 'studyName': 'The Single cell transcriptomic analysis of PBMCs in Extreme Longevity',
 'project': 'ILO',
 'dataTypeAll': ['Gene Expression'],
 'species': ['Human'],
 'specimenType': ['Blood'],
 'dataContributor': ['Tufts Medical Center'],
 'studyAbstract': 'This study provides data from 7 centenarian samples (> 100 years) and 2 younger control samples (20-59 years) from New England Centenarian Study (NECS) at Boston University and the Integrative Longevity Omics (ILO). Peripheral Blood Mononuclear Cells (PBMCs) transcriptional and protein expression were profiled at a single cell resolution. Pluripotent stem cells were also generated. Droplet-based single cell CITE-seq data (16,082 cells).',
 'studyDescription': 'syn26067509',
 'studyFocus': ['Longevity'],
 'program': ['ELITE'],
 'grantNumber': ['UH2AG064704'],
 'studyWiki': 'syn53366748',
 'acknowledgement': 'syn27229419/wiki/621602',
 'methods': ['syn26133757'],
 'relatedStudies': ['syn26066886',
  'syn26969

In [29]:
applied_annos = syn.set_annotations("syn53366748", new_annos)

[WARNING] /var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_5897/1454500765.py:1: DeprecationWarning: Call to deprecated method setAnnotations. (deprecated and replaced with `set_annotations` This method is UNSAFE and may overwrite existing annotations without confirming that you have retrieved and updated the latest annotations) -- Deprecated since version 2.1.0.
  applied_annos = syn.setAnnotations("syn53366748", new_annos)



In [34]:
# add annotations to elpscrna_metadata
entity_id = "syn53366749"

annos = {
    "studyKey": "ELPSCRNA",
    "project": "ILO",
    "dataTypeAll": ["metadata"],
    "dataSubtype": "raw",
    "fileFormat": "CSV",
    "species": ["Human"],
    "resourceType": 'metadata',
    "metadataType": 'folder',
    "program": ["ELITE"],
    "grantNumber": ["UH2AG064704"],
    "isModelSystem": False,
    "isReleased": False,
}

applied_annos = syn.setAnnotations(entity_id, annos)

[WARNING] /var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_5897/1168609817.py:19: DeprecationWarning: Call to deprecated method setAnnotations. (deprecated and replaced with `set_annotations` This method is UNSAFE and may overwrite existing annotations without confirming that you have retrieved and updated the latest annotations) -- Deprecated since version 2.1.0.
  applied_annos = syn.setAnnotations(entity_id, annos)



In [35]:
# add annotations to elpscrna_metadata
entity_id = "syn53366747"

annos = {
    "studyKey": "ELPSCRNA",
    "project": "ILO",
    "dataType": ["documentation"],
    "dataSubtype": "raw",
    "species": ["Human"],
    "resourceType": "metadata",
    "metadataType": "documentation",
    "program": ["ELITE"],
    "grantNumber": ["UH2AG064704"],
    "isModelSystem": False,
    "isReleased": False,
}

applied_annos = syn.setAnnotations(entity_id, annos)

[WARNING] /var/folders/p0/5m4pdsm55jn_d5nzbjv6bjf40000gq/T/ipykernel_5897/424773228.py:18: DeprecationWarning: Call to deprecated method setAnnotations. (deprecated and replaced with `set_annotations` This method is UNSAFE and may overwrite existing annotations without confirming that you have retrieved and updated the latest annotations) -- Deprecated since version 2.1.0.
  applied_annos = syn.setAnnotations(entity_id, annos)

